<a href="https://colab.research.google.com/github/PARULCHUTANIPC/parul/blob/p1/fmnist_d3%2C4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
torch.set_printoptions(linewidth= 120)
torch.set_grad_enabled(True)
from torch.utils.tensorboard import SummaryWriter

In [10]:

def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

ERROR! Session/line number was not unique in database. History logging moved to new session 60


In [0]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        # implement the forward pass
        # 1) Input layer
        t=t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # 6) Output Layer
        t = self.out(t)
#t = F.softmax(t, dim=1)

        return t

In [0]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

# TRAINING WITH A SINGLE BATCH

In [0]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set
   ,batch_size=100)

optimizer = optim.Adam(network.parameters(),lr = 0.01)

# Obtain the batch 
batch = next(iter(train_loader))
images, labels = batch


# PASS THE BATCH IN THE NETWORK  --------  network is the name of the class we made..... in which we defined the layers, 1 input, 2 conv, 2 linear and 1 output layer
preds = network(images)

#CALCULATE LOSS
loss = F.cross_entropy(preds, labels)

# CALCULATE GRADIENTS
loss.backward()

# UPDATE WEIGHTS
optimizer.step()


In [14]:
print ('loss1 : ', loss.item()) # it will give the loss when the images are given to the network  first time
# Now, the images are again passed to the network and calculate the loss again and compare
preds = network(images)
loss = F.cross_entropy(preds, labels)
print('loss2 : ', loss.item())


loss1 :  2.2978203296661377
loss2 :  2.2623584270477295


**# TRAINING WITH ALL BATCHES / ONE EPOCH**

In [0]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set
   ,batch_size=100)

optimizer = optim.Adam(network.parameters(),lr = 0.01)

# THIS IS THE SAME AS TRAINING A SINGLE BATCH. EXCEPT TO THAT , WE HAVE TO APPLY A LOOP SO THAT ALL BATCHES CAN PASS THROUGH THE NETWORK TO GET TRAINED
#####---------------------------
total_loss= 0
total_correct = 0


# Obtain the batch 
#batch = next(iter(train_loader))
#images, labels = batch

# SELECTING ALL BATCHES

for batch in train_loader:

# PASS THE BATCH IN THE NETWORK  --------  network is the name of the class we made..... in which we defined the layers, 1 input, 2 conv, 2 linear and 1 output layer
  preds = network(images)

#CALCULATE LOSS
  loss = F.cross_entropy(preds, labels)


# -------------------- EXTRA LINE ---- SET THE INITIAL GRADIENT ZERO AFTER EVERY ITERATION, OTHERWISE IT WILL ADD UP THE WEIGHT AFTER EVERY STEP WHICH WILL ULTIMATELY RESULT IN BAD ACCURACY, HIGHER LOSS

  optimizer.zero_grad()

# CALCULATE GRADIENTS
  loss.backward()

# UPDATE WEIGHTS  
  optimizer.step()

  ##----------- Now, add the loss calculated  and the correct predictions done for every batches
  total_correct+= get_num_correct(preds, labels)
  total_loss+= loss.item()

In [16]:
print("epoch : ", 0, "Correct Predictions : ", total_correct, "   Total_Loss : ", total_loss)
accuracy = total_correct/len(train_set)
print ("Accuracy : ", accuracy)

epoch :  0 Correct Predictions :  58477    Total_Loss :  40.48264418143481
Accuracy :  0.9746166666666667


# TRAIN THE IMAGES MULTIPLE TIMES / multiple epochs

In [17]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set
   ,batch_size=100)

optimizer = optim.Adam(network.parameters(),lr = 0.01)

# Obtain the batch 
batch = next(iter(train_loader))
images, labels = batch

################ ADDING THE TENSORBOARD  ########&&&&&&&&&&&&&&&&&&&&&
grid = torchvision.utils.make_grid(images)

tb= SummaryWriter()
tb.add_image('images', grid)
tb.add_graph(network, images)

######################            CREATE A LOOOPPPPPPP ##########################################


for epoch in range(3):


# THIS IS THE SAME AS TRAINING A SINGLE BATCH. EXCEPT TO THAT , WE HAVE TO APPLY A LOOP SO THAT ALL BATCHES CAN PASS THROUGH THE NETWORK TO GET TRAINED
#####---------------------------
  total_loss= 0
  total_correct = 0




# SELECTING ALL BATCHES

  for batch in train_loader:

# PASS THE BATCH IN THE NETWORK  --------  network is the name of the class we made..... in which we defined the layers, 1 input, 2 conv, 2 linear and 1 output layer
    preds = network(images)

#CALCULATE LOSS
    loss = F.cross_entropy(preds, labels)


# -------------------- EXTRA LINE ---- SET THE INITIAL GRADIENT ZERO AFTER EVERY ITERATION, OTHERWISE IT WILL ADD UP THE WEIGHT AFTER EVERY STEP WHICH WILL ULTIMATELY RESULT IN BAD ACCURACY, HIGHER LOSS

    optimizer.zero_grad()

# CALCULATE GRADIENTS
    loss.backward()

# UPDATE WEIGHTS  
    optimizer.step()

  ##----------- Now, add the loss calculated  and the correct predictions done for every batches
    total_correct+= get_num_correct(preds, labels)
    total_loss+= loss.item()

###-------------------ADDING THE TENSORBOARD--------------------------------
  tb.add_scalar('Loss', total_loss, epoch)
  tb.add_scalar('Correct Predictions', total_correct, epoch)
  tb.add_scalar('Accuracy', total_correct/len(train_set), epoch)
  tb.add_histogram('Conv1.bias', network.conv1.bias, epoch)
  tb.add_histogram('Conv1.weight', network.conv1.weight, epoch)
  tb.add_histogram('Conv1.weight.grad', network.conv1.weight.grad, epoch)


  print("epoch : ", epoch, "Correct Predictions : ", total_correct, "   Total_Loss : ", total_loss)
tb.close()

ERROR! Session/line number was not unique in database. History logging moved to new session 61


epoch :  0 Correct Predictions :  58474    Total_Loss :  39.346368390321004
epoch :  1 Correct Predictions :  60000    Total_Loss :  0.0062781407582406246
epoch :  2 Correct Predictions :  60000    Total_Loss :  0.002183776653737368


In [0]:
#print("epoch : ", epoch, "Correct Predictions : ", total_correct, "   Total_Loss : ", total_loss)
accuracy = total_correct/len(train_set)
print ("Accuracy : ", accuracy*100,"%")

#ANALYZING CNN RESULTS / CONFUSION MATRIX

In [0]:
print(len(train_set))
len(train_set.targets)

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np
#from resources.plotcm import plot_confusion_matrix

In [0]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set
   ,batch_size=100)

optimizer = optim.Adam(network.parameters(),lr = 0.01)



######################            CREATE A LOOOPPPPPPP ##########################################


for epoch in range(5):


# THIS IS THE SAME AS TRAINING A SINGLE BATCH. EXCEPT TO THAT , WE HAVE TO APPLY A LOOP SO THAT ALL BATCHES CAN PASS THROUGH THE NETWORK TO GET TRAINED
#####---------------------------
  total_loss= 0
  total_correct = 0
  total_pred = []
  
# Obtain the batch 
#batch = next(iter(train_loader))
#images, labels = batch

# SELECTING ALL BATCHES

  for batch in train_loader:

# PASS THE BATCH IN THE NETWORK  --------  network is the name of the class we made..... in which we defined the layers, 1 input, 2 conv, 2 linear and 1 output layer
    preds = network(images)

#CALCULATE LOSS
    loss = F.cross_entropy(preds, labels)


# -------------------- EXTRA LINE ---- SET THE INITIAL GRADIENT ZERO AFTER EVERY ITERATION, OTHERWISE IT WILL ADD UP THE WEIGHT AFTER EVERY STEP WHICH WILL ULTIMATELY RESULT IN BAD ACCURACY, HIGHER LOSS

    optimizer.zero_grad()

# CALCULATE GRADIENTS
    loss.backward()

# UPDATE WEIGHTS  
    optimizer.step()

  ##----------- Now, add the loss calculated  and the correct predictions done for every batches
    total_correct+= get_num_correct(preds, labels)
    total_loss+= loss.item()
    total_pred.append(preds)
    
  print("epoch : ", epoch, "Correct Predictions : ", total_correct, "   Total_Loss : ", total_loss)

In [0]:
len(total_pred)

In [0]:
############################## Code by Soumith#######################
prediction_list=[]
def predict(self, dataloader):
  for i, batch in enumerate(dataloader):
    pred,output  = self.step(batch)
    prediction_list.append(pred.cpu())

In [0]:
predict()

In [0]:
#type(total_pred)
len(train_set.targets)
train_preds

In [0]:
print(len(total_pred[1]))
print(len(total_pred[1][1]))

In [0]:
listt = []
for i in total_pred:
  for j in i:
    listt.append(j.argmax(dim=0).tolist())

In [0]:
total_pred[1].argmax(dim=0)

In [0]:
print(len(listt))
print(type(listt))
type(listt[0])

In [0]:
labels_to_compare = []
for i in train_set.train_labels:
  labels_to_compare.append(i.tolist())

In [0]:
print(type(labels_to_compare))
print(type(labels_to_compare[0]))
len(labels_to_compare)

In [0]:
cm = confusion_matrix(labels_to_compare, listt)
#print(type(cm))
#cm.dtype
cm

In [0]:
labels_to_compare == listt

In [0]:
not_same = 0
for i in range(len(listt)):
  if listt[i] != labels_to_compare[i]:
    not_same+= 1


In [0]:
not_same

In [0]:
s1=[1,1,1]
s2=[1,1,1]
s1==s2

In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
cm = confusion_matrix(labels_to_compare, listt)
cm

In [0]:
sns.heatmap(data = cm)

In [0]:
 names = (
    'T-shirt/top'
    ,'Trouser'
    ,'Pullover'
    ,'Dress'
    ,'Coat'
    ,'Sandal'
    ,'Shirt'
    ,'Sneaker'
    ,'Bag'
    ,'Ankle boot'
)
plt.figure(figsize=(10,10))
#plt.plot(data = cm)

#plot_confusion_matrix(cm, names)

In [0]:
from sklearn.metrics import classification_report

In [0]:
print(classification_report(labels_to_compare, listt))

In [0]:
for i in range(len(listt)):
  labels_to_compare[i]==listt[i]

In [0]:
with torch.no_grad():
    prediction_loader = torch.utils.data.DataLoader(train_set, batch_size=10000)
    train_preds = get_all_preds(network, prediction_loader)